# Using textual analysis of financial documents to gain insight on a company's market performance
Individual student final project, Physics 250 Econophysics, Winter 2017

Dustin Burns, PhD Candidate, UC Davis

## Intro

The goal of this study is to measure the correlation between negative tone in a company's SEC filing reports and the short-term market return of the company's stock, reproducing the results found by [Loughran and McDonald](https://www.uts.edu.au/sites/default/files/ADG_Cons2015_Loughran%20McDonald%20JE%202011.pdf).

Tone is measured in the documents using a "bag-of-words" algorithm, counting the number of negative words in the document which appear in a custom finance dictionary, defined [here](http://www3.nd.edu/~mcdonald/Word_Lists_files/LoughranMcDonald_MasterDictionary_2014.
xlsx).

The list of data files and metadata used to crosscheck this analysis can be found [here](http://www3.nd.edu/~mcdonald/Data/LoughranMcDonald_10X_2014.xlsx), and the directory structure of the SEC Edgar database is described [here](https://www.sec.gov/edgar/searchedgar/accessing-edgar-data.htm).

## Setup

In [40]:
import pandas as pd
from yahoo_finance import Share
import argparse
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy.stats as stats
import wget
import nltk
import re
import os
from collections import Counter
from datetime import timedelta
from collections import OrderedDict
import random

## Load dictionary

In [41]:
dictionary = pd.read_excel('data/LoughranMcDonald_MasterDictionary_2014.xlsx', sheetname=0)
fin_neg = dictionary.Word[dictionary.Negative > 0].values

In [42]:
# What are some example negative words?
print random.sample(fin_neg, 30)

[u'UNCOVER', u'DISAPPEARANCES', u'DISMISSES', u'PANIC', u'DEADWEIGHT', u'EXPLOITATIONS', u'CLOSED', u'UNREIMBURSED', u'INCIDENCE', u'UNCOLLECTED', u'RECALLS', u'TERMINATION', u'ABDICATION', u'UNJUST', u'CONFINING', u'SUMMONSES', u'ERRATIC', u'INORDINATELY', u'INSUBORDINATION', u'EXACERBATE', u'INACTIVATIONS', u'FELONIES', u'MISAPPROPRIATING', u'UNDERMINE', u'CATASTROPHIC', u'CONFESSED', u'OVERLOAD', u'OVERAGES', u'DEADLOCK', u'CONCILIATING']


## Load and clean metadata

In [43]:
# Load first 10 rows for demo
data = pd.read_excel('data/LoughranMcDonald_10X_2014_test_1.xlsx', sheetname=0, skip_header=1)

# Load N randomly sampled rows for full study
#N = 5000
#data = pd.read_excel('data/LoughranMcDonald_10X_2014.xlsx', sheetname=0, skip_header=1)
#data = data.loc[random.sample(list(data.index), N)]

# Drop unused cols, copy cols to crosscheck
data = data.drop(['N_Positive', 'N_Uncertainty', 'N_Litigious', 'N_WeakModal', 'N_StrongModal', 'N_Constraining', 'N_Negation', 'GrossFileSize', 'NetFileSize', 'ASCIIEncodedChars', 'HTMLChars', 'XBRLChars', 'TableChars'], axis=1)
data['N_Words_Paper'] = data.N_Words
data['N_Unique_Words_Paper'] = data.N_Unique_Words
data['N_Negative_Paper'] = data.N_Negative
data = data.drop(['N_Negative'], axis=1)

In [44]:
# Convert filing date to datetime
data.FILING_DATE  = data.FILING_DATE.astype('string')
for time in data.FILING_DATE:
  t = time[0:4] + '-' + time[4:6] + '-' + time[6:8]
  data.ix[data.FILING_DATE == time, 'FILING_DATE'] = t
data.FILING_DATE  = pd.to_datetime(data.FILING_DATE)

# End date is 3 days after filing date
data['End_Date'] = data.FILING_DATE + timedelta(days=3)

# Skip weekends
#print data.FILING_DATE
#print data.End_Date
#
#delta = timedelta(days=1)
#for ind, date in enumerate(data.End_Date):
#  i=0
#  while i < 4:
#    if date.weekday() not in [5, 6]:
#      print data.End_Date[ind].weekday()
#      data.End_Date[ind] = data.End_Date[ind] + delta
#      i += 1 

## Scrape and clean data files

In [45]:
# Helper functions:

# Return counts of words in dictionary
def count_words(toks, dictionary):
  cnts = [0]*len(dictionary)
  for tok in toks:
    if tok.upper() in dictionary: cnts[dictionary.tolist().index(tok.upper())] += 1
  return cnts

In [46]:
# Analysis arrays
toks_raw_dirty = []
toks_raw_clean = []
toks_total = []
toks_unique = []
toks_freq = []
avg_freq = []
neg_counts = []
neg_total = []

# Loop through metadata rows
for name in data.FILE_NAME:
  
  # Build file name
  fname = 'https://www.sec.gov/Archives/edgar/data/' + name.split('data_')[1].replace('_','/')
  data.ix[data.FILE_NAME == name, 'FILE_NAME'] = fname
  
  # Download file
  try: 
    f = 'data/10X/' + fname.split('/')[-1]
  #  f = wget.download(fname, './data/10X')
  except: continue
  
  # Raw toks before cleaning
  text = open(f).read()
  toks_dirty = sorted(np.array(nltk.word_tokenize(text)))
  toks_raw_dirty.append(toks_dirty)

  # Remove non-text HTML blocks
  # TO DO: compile regex before loop to speed up
  os.rename(f, f + '.orig')
  with open(f + '.orig', 'rb') as fin, open(f, 'wb') as fout:
    text = fin.read()
    text = re.sub(r'<IMS-HEADER>.*?</IMS-HEADER>', '', text, flags=re.DOTALL)
    text = re.sub(r'<SEC-HEADER>.*?</SEC-HEADER>', '', text, flags=re.DOTALL)
    text = re.sub(r'<GRAPHIC>.*?</GRAPHIC>', '', text, flags=re.DOTALL)
    text = re.sub(r'<ZIP>.*?</ZIP>', '', text, flags=re.DOTALL)
    text = re.sub(r'<EXCEL>.*?</EXCEL>', '', text, flags=re.DOTALL)
    text = re.sub(r'<PDF>.*?</PDF>', '', text, flags=re.DOTALL)
    text = re.sub(r'<XBRL>.*?</XBRL>', '', text, flags=re.DOTALL)
    text = re.sub(r'<.*?>', '', text, flags=re.DOTALL)
    fout.write(text)
  os.remove(f + '.orig')

  text = open(f).read()
  try: toks = np.array(nltk.word_tokenize(text))
  except: continue
  
  # Remove words that contain numbers or special characters
  special_inds = [not bool(re.search('[\d\/\*\'\-,=;:@<>\.\_]', x)) for x in toks]  
  try: toks = toks[np.array(special_inds)]
  except: pass

  # Remove single char words
  word_inds = [len(x)>1 for x in toks]
  toks = toks[np.array(word_inds)]
  
  # Remove negated words here for positive words
  # ['no', 'not', 'none', 'neither', 'never', 'nobody'] occurs four or fewer words before
   
  # Alphabetize
  toks = sorted(toks) 

  # Fill total word counts
  toks_raw_clean.append(toks)
  toks_total.append(len(toks))
  #data.ix[data.FILE_NAME == fname, 'N_Words'] = len(toks) 
    
  # Fill unique word counts
  toks_dict = OrderedDict(sorted(Counter(toks).items()))
  toks_freq.append(toks_dict.values())
  avg_freq.append(np.nanmean(toks_dict.values()))
  toks_unique.append(toks_dict.keys())
  #data.ix[data.FILE_NAME == fname, 'N_Unique_Words'] = len(toks_dict.keys()) 
  
  # Fill negative word counts
  neg_in_dict = count_words(toks, fin_neg)
  neg_counts.append(neg_in_dict)
  neg_total.append(sum(neg_in_dict))

data['Toks_Dirty'] = toks_raw_dirty
data['Toks_Raw'] = toks_raw_clean
data['Toks_Total'] = toks_total
data['Toks_Freq'] = toks_freq
data['Avg_Freq'] = avg_freq
data['Toks_Unique'] = toks_unique
data['Neg_Counts'] = neg_counts
data['Neg_Total'] = neg_total

print data.Toks_Dirty
print data.Toks_Unique

0    [$, $, $, $, $, $, $, $, $, $, $, $, $, $, $, ...
1    [$, $, $, $, $, $, $, $, $, $, $, $, $, $, $, ...
2    [$, $, $, $, $, $, $, $, $, $, $, $, $, $, $, ...
3    [$, $, $, $, $, $, $, $, $, $, $, $, $, $, $, ...
4    [$, $, $, $, $, $, $, $, $, $, $, $, $, $, $, ...
5    [$, $, $, $, $, $, $, $, $, $, $, $, $, $, $, ...
6    [#, #, #, $, $, $, $, $, $, $, $, $, $, $, $, ...
7    [$, $, $, $, $, $, $, $, $, $, $, $, $, $, $, ...
8    [$, $, $, $, $, $, $, $, $, $, $, $, $, $, $, ...
9    [!, !, !, !, $, $, $, $, $, $, $, $, $, $, $, ...
Name: Toks_Dirty, dtype: object
0    [ACT, ALFERONR, AMENDMENT, AND, ANNUAL, ASSETS...
1    [ALFERONR, AMENDMENT, ANALYSIS, AND, ASSETS, A...
2    [ACT, ACTIVITIES, ADJUSTED, ANALYSIS, AND, APP...
3    [ACT, ANALYSIS, AND, ASSETS, Accounts, Act, Ad...
4    [ACCOUNTANTS, ACT, AID, AND, APB, ASSETS, AUDI...
5    [ACCOUNTANTS, ACCOUNTING, ACT, ACTIVITIES, AL,...
6    [ACT, AGGREGATED, AGREEMENTS, AND, ANNUAL, AT,...
7    [ABOVE, ACCOUNTANTS, ACCOUNT

In [47]:
print "Total words crosscheck"
print data.Toks_Total.values
print data.N_Words_Paper.values
print
print "Negative words crosscheck"
print data.Neg_Total.values
print data.N_Negative_Paper.values

Total words crosscheck
[13159  3423  2231  1851  2428  5127  3303 31494  2938  2858]
[11222  2705  1698  1206  1783  4163  2347 25479  2310  2252]

Negative words crosscheck
[153  24  21  26  11  98  21 267  50  10]
[142  21  21   7  11  90  21 240  43   8]


In [48]:
# Calculate proportional yield, weighting negative word count by document length
data['Prop_Yield'] = data.Neg_Total / data.Toks_Total
data['Prop_Yield_Paper'] = data.N_Negative_Paper / data.N_Words_Paper

# Calculate tf.idf yield, weighting word count by term frequency and inverse document frequency
# number of documents containing 1+ occurance of ith word
df = [0] * len(dictionary)
for cnts in data.Neg_Counts:
  for i, cnt in enumerate(cnts):
    if cnt > 0: df[i] += 1
# Number of documents
N = len(data.CIK.values)
dict_len = len(fin_neg)
# Calculate sum of tf.idf weights
w = [0] * N
for j in range(0, N):
  weights = [0] * dict_len
  for i in range(0, dict_len):
    tf = data.Neg_Counts.values[j][i]
    if tf > 0:
      weights[i] = (1 + np.log(tf)) * np.log(N / df[i]) / (1 + np.log(data.Avg_Freq[j]))
  w[j] = sum(weights)
data['Neg_Yield'] = w

## Scrape historical stock prices
Historical stock data is acquired from the Yahoo Finance database with the python plugin [yahoo-finance](https://github.com/lukaszbanasiak/yahoo-finance).

The key for converting SEC CIK number to ticker symbol is obtained [here](http://rankandfiled.com/#/data/tickers).

In [49]:
# Helper function to convert CIK number to ticker symbol
key = pd.read_csv('data/cik_ticker.csv', sep='|')  
def cik_to_ticker(cik):
  key.CIK = key.CIK.astype('str')
  try: 
    ind = key.CIK[key.CIK == cik].index.tolist()[0]
    ticker = key.Ticker[ind]
  except IndexError: 
    ticker = 'NaN'
  return ticker

# Convert CIK to ticker symbol, delete row if missing from key
nan_ind = []
tickers = []
for ind, cik in enumerate(data.CIK): 
  ticker = cik_to_ticker(str(cik))
  if ticker == 'NaN': nan_ind.append(ind)
  else: tickers.append(ticker)
data = data.drop(data.index[nan_ind])
data['Ticker'] = tickers
data = data.reset_index(drop=True)

In [50]:
# Get historical returns from yahoo_finance API 
filing_return = []
market_return = []
for ind, ticker in enumerate(data.Ticker):
  share = Share(ticker)
  market = Share('^GSPC')
  try: 
    returns = share.get_historical(str(data.FILING_DATE[ind])[0:10], str(data.End_Date[ind])[0:10])
    market = market.get_historical(str(data.FILING_DATE[ind])[0:10], str(data.End_Date[ind])[0:10])
  except: 
    filing_return.append(float('NaN'))
    market_return.append(float('NaN'))
    continue
  returns  = pd.DataFrame(returns)
  returns.Adj_Close = returns.Adj_Close.astype(float)
  filing_return.append( ((returns.Adj_Close.tail(1).values - returns.Adj_Close.head(1).values) / returns.Adj_Close.head(1).values)[0] )
  market = pd.DataFrame(market)
  market.Adj_Close = market.Adj_Close.astype(float)
  market_return.append( ((market.Adj_Close.tail(1).values - market.Adj_Close.head(1).values) / market.Adj_Close.head(1).values)[0])
data['Filing_Return'] = filing_return
data['Market_Return'] = market_return
data['Excess_Return'] = data.Filing_Return - data.Market_Return

In [51]:
# Clean any rows with missing data
data = data.dropna()
data = data.reset_index(drop=True)
  
# Write final dataframe to file
data.to_csv(path_or_buf='data/cleaned_10X_test.csv') 

## Statistics and visualizations

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

#data1 = pd.read_csv('data/cleaned_10X_1.csv')
 
(a, b, r, tt, stderr) = stats.linregress(data.Neg_Yield, data.Excess_Return)
print 'Slope:           ' + str(a)
print 'Standard error:  ' + str(stderr)
print 'R-squared:       ' + str(r**2)
print 'p-value:         ' + str(tt)

plt.figure()
#ax = sns.regplot(x="Prop_Yield_Paper", y="Excess_Return", data=data, x_bins=5, ci=68)
ax = sns.regplot(x="N_Negative_Paper", y="Excess_Return", data=data, x_bins=5, ci=68, x_estimator=np.median)
#plt.xlabel('Weighted negative word count', horizontalalignment='right', x=1.0, fontsize=16)
plt.xlabel('Unweighted negative word count', horizontalalignment='right', x=1.0, fontsize=16)
#plt.xlabel('Proportional weighted negative word count', horizontalalignment='right', x=1.0, fontsize=16)
#plt.ylabel('Filing period excess return (%)', horizontalalignment='right', y=1.0, fontsize=16)
plt.ylabel('Mean filing period excess return (%)', horizontalalignment='right', y=1.0, fontsize=16)
plt.show()

## Product ideation

My results are consistent with those found by [Loughran and McDonald](https://www.uts.edu.au/sites/default/files/ADG_Cons2015_Loughran%20McDonald%20JE%202011.pdf): negative tone in SEC filings is weakly correlated with the filing period excess return, but not enough to completely inform a trading strategy.

However, a strong negative sentiment of a company's filings CAN be used to red flag potential weaknesses in a portfolio. Below, I outline a product using the techniques shown above, to be used by investors to inform their portfolio management decision making:

* For each company's stock in portfolio, periodically check for SEC filing
* If recent filing exists, calculate negative word count
* If negative word count above threshold, red flag company for further investigation
